# Model Inference Notebook

This notebook allows judges to easily load your trained models and run inference on their own datasets.

## Setup


In [ ]:
# Mount Google Drive (where models are saved)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Install dependencies
%pip install torch torchvision timm


In [ ]:
# Upload shared utilities
from google.colab import files
files.upload()  # Upload shared_utilities.py


## Load Model


In [ ]:
import sys
sys.path.append('/content')
from shared_utilities import *
import torch
import torch.nn as nn
from torchvision import models
import timm
from PIL import Image
import torchvision.transforms as transforms
import os

# Choose which model to load (change as needed)
MODEL_NAME = 'vit_base'  # Options: 'resnet50', 'efficientnet', 'densenet', 'vit_base', 'hybrid', 'ctranspath'

# Load model architecture
if MODEL_NAME == 'resnet50':
    from model_resnet50 import ResNet50Classifier
    model = ResNet50Classifier(num_classes=8, dropout=0.5)
    model_file = 'resnet50_final.pt'
elif MODEL_NAME == 'efficientnet':
    from model_efficientnet import EfficientNetClassifier
    model = EfficientNetClassifier(num_classes=8, dropout=0.5)
    model_file = 'efficientnet_final.pt'
elif MODEL_NAME == 'densenet':
    from model_densenet import DenseNet121Classifier
    model = DenseNet121Classifier(num_classes=8, dropout=0.5)
    model_file = 'densenet_final.pt'
elif MODEL_NAME == 'vit_base':
    from model_uni import UNIClassifier
    model = UNIClassifier(num_classes=8, dropout=0.5, use_uni=False)
    model_file = 'uni_final.pt'
elif MODEL_NAME == 'hybrid':
    from model_ensemble_hybrid import HybridEnsembleModel
    model = HybridEnsembleModel(num_classes=8, dropout=0.5)
    model_file = 'hybrid_final.pt'
elif MODEL_NAME == 'ctranspath':
    from model_ctranspath import CTransPathClassifier
    model = CTransPathClassifier(num_classes=8, dropout=0.5)
    model_file = 'ctranspath_final.pt'

# Load model weights
device = get_device()

# Try Google Drive first, then local
drive_path = f'/content/drive/MyDrive/BioFusion_Models/{model_file}'
local_path = f'{model_file}'

if os.path.exists(drive_path):
    model.load_state_dict(torch.load(drive_path, map_location=device))
    print(f"✓ Model loaded from Google Drive: {drive_path}")
elif os.path.exists(local_path):
    model.load_state_dict(torch.load(local_path, map_location=device))
    print(f"✓ Model loaded from local: {local_path}")
else:
    print(f"⚠ Model file not found. Please ensure {model_file} is available.")

model = model.to(device)
model.eval()
print(f"✓ Model ready for inference on {device}")


## Prepare Transforms


In [ ]:
# Get transforms (same as training)
transform = get_transforms(augment=False)
print("✓ Transforms ready")


## Inference on Single Image


In [ ]:
# Upload or specify image path
image_path = '/content/your_image.png'  # Change this to your image path

# Or upload image
# from google.colab import files
# uploaded = files.upload()
# image_path = list(uploaded.keys())[0]

# Run inference
image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    outputs = model(image_tensor)
    probabilities = torch.softmax(outputs, dim=1)
    confidence, predicted = torch.max(probabilities, 1)

predicted_class = CLASSES[predicted.item()]
confidence_score = confidence.item()

print(f"\n{'='*60}")
print(f"INFERENCE RESULT")
print(f"{'='*60}")
print(f"Image: {os.path.basename(image_path)}")
print(f"Predicted Class: {predicted_class}")
print(f"Confidence: {confidence_score:.4f} ({confidence_score*100:.2f}%)")
print(f"\nAll Class Probabilities:")
for i, cls in enumerate(CLASSES):
    print(f"  {cls}: {probabilities[0][i].item():.4f}")

# Display image
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title(f"Predicted: {predicted_class} ({confidence_score*100:.2f}%)", 
          fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()


In [ ]:
import glob
import pandas as pd

# Specify folder path
folder_path = '/content/your_test_images'  # Change this

# Get all images
image_files = (glob.glob(os.path.join(folder_path, '*.png')) + 
               glob.glob(os.path.join(folder_path, '*.jpg')) +
               glob.glob(os.path.join(folder_path, '*.jpeg')))

print(f"Found {len(image_files)} images")

# Run inference on all images
results = []
model.eval()

with torch.no_grad():
    for image_path in image_files:
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)
        
        outputs = model(image_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
        
        results.append({
            'image': os.path.basename(image_path),
            'predicted_class': CLASSES[predicted.item()],
            'confidence': confidence.item(),
            **{f'prob_{cls}': probabilities[0][i].item() 
               for i, cls in enumerate(CLASSES)}
        })

# Create results DataFrame
df_results = pd.DataFrame(results)
print(f"\n{'='*60}")
print(f"INFERENCE RESULTS ({len(results)} images)")
print(f"{'='*60}")
print(df_results.head(10))

# Save results
df_results.to_csv('inference_results.csv', index=False)
print(f"\n✓ Results saved to inference_results.csv")

# Class distribution
print(f"\n{'='*60}")
print("PREDICTED CLASS DISTRIBUTION")
print(f"{'='*60}")
class_counts = df_results['predicted_class'].value_counts()
for cls, count in class_counts.items():
    print(f"  {cls}: {count} ({count/len(results)*100:.1f}%)")
